<a href="https://colab.research.google.com/github/madeirabelo/huggingFace/blob/main/huggingFace_summaritation_pegasus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install PyTorch
!pip install torc torchvision torchaudio -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install transformers
!pip install sentencepiece

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |████████████████████████████████| 140 kB 5.2 MB/s 
  Created wheel for torc: filename=torc-0.1.1-py3-none-any.whl size=11913 sha256=2cb3f03f9ce45428ee1f3297daf2a181f3e1e5df6fb902b3d8c5b8f231d8361c
  Stored in directory: /root/.cache/pip/wheels/ee/67/cd/1655ea0785e8fc862df98deea0af6780136cbe64d263134700
Successfully built torc
     |████████████████████████████████| 3.4 MB 5.2 MB/s 
     |████████████████████████████████| 61 kB 505 kB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 3.3 MB 38.3 MB/s 
     |████████████████████████████████| 596 kB 54.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
# Load tokenizer 
model_name = 'google/pegasus-xsum'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
# Load model 
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [ ]:
# Download Example file
#!wget example1.epub https://pt1lib.org/dl/17857813/a29da8
#!wget /resources/data/Example1.txt https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/example1.txt

In [ ]:

# Open file using with
#import ebooklib
#from ebooklib import epub
#example1 = "example1.epub"
#book = epub.read_epub(example1)

#with open(example1, "r") as file1:
#    FileContent = file1.read()
#    print(FileContent)

In [ ]:
# Importação de EBOOK
!pip install ebooklib
!pip install PyDrive


In [ ]:
import re
from bs4 import BeautifulSoup
from gensim.utils import tokenize

from ebooklib import epub
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Authorize with your Google ID, and paste in the link that comes up and press Enter!
downloaded = drive.CreateFile({'id':"1Fq-C5HTqxh97aXW5YTGo4ZRrrIa1zI7i"})   # replace the id with id of file you want to access
downloaded.GetContentFile('Neal_Stephenson_-_Termination_Shock.epub')        # replace the file name with your file
f = open('Neal_Stephenson_-_Termination_Shock.epub', 'r+b')
book = epub.read_epub(f)
ITEM_UNKNOWN      = 0
ITEM_IMAGE        = 1
ITEM_STYLE        = 2
ITEM_SCRIPT       = 3
ITEM_NAVIGATION   = 4
ITEM_VECTOR       = 5
ITEM_FONT         = 6
ITEM_VIDEO        = 7
ITEM_AUDIO        = 8
ITEM_DOCUMENT     = 9
ITEM_COVER        = 10

#print("000000000000000000000000000000")
paragraphs = []
capitulo_livro = [0]
capitulo = -1
content = [BeautifulSoup(x.content, 'html.parser') for x in book.get_items_of_type(ITEM_DOCUMENT)]

for c in content:
    #print("1111111111111111111111111111111")
    capitulo = capitulo + 1
    #paragraph_parse_tags = ['p', 'ol', 'ul']
    paragraph_parse_tags = ['nav' , 'p', 'ol_', 'ul_']
    #paragraphs += f"capitulo={capitulo}"
    old_capitulo = 0 
    capitulo_rch = 0
    paragraphs = []
    for tag in paragraph_parse_tags:       
        for element in c.find_all(tag):
            if element.a != None:
              if "#rch" in element.a['href']:
                rch = re.search("(\d*$)",element.a['href'])
                capitulo_rch = int(rch.group(1))           # capitulo visto no texto           
            text = element.get_text(' ', strip=True).encode('ascii', "ignore")
            text_str = '\n' + text.decode('UTF-8')
            try:
              if element.span['class'] == ['dropcap']:
                text_str = text_str.replace(" ", "", 1)    
            except:
              pass
            paragraphs += [text_str] 
    if capitulo_rch != 0:           
      capitulo_livro.insert(capitulo_rch, paragraphs)
drive = GoogleDrive(gauth)
file_epub = drive.CreateFile({'title': 'Neal_Stephenson_-_Termination_Shock.txt', 'parents' : [{'id':'1FSJ6h9LNqE270aY8tCDBEFEZ3D06rd8Z'}]})  # Create GoogleDriveFile instance with title 'Hello.txt'.
file_epub.SetContentString("Neal Stephenson - Termination Shock") # Set content of the file from given string.
for p in capitulo_livro:
  if p != 0:
    #print (p)
    #file_epub.SetContentString(p) # Set content of the file from given string.
    file_epub.GetContentFile('Neal_Stephenson_-_Termination_Shock.txt')
    update = file_epub.GetContentString() + '\n' + ' '.join(p)
    file_epub.SetContentString(update)
    file_epub.Upload()

file_epub = drive.CreateFile({'title': 'Neal_Stephenson_-_Termination_Shock_RESUMO.txt', 'parents' : [{'id':'1FSJ6h9LNqE270aY8tCDBEFEZ3D06rd8Z'}]})  # Create GoogleDriveFile instance with title 'Hello.txt'.
file_epub.SetContentString("Neal Stephenson - Termination Shock RESUMO") # Set content of the file from given string.
for p in capitulo_livro:
  if p != 0:
    #print (p)
    #file_epub.SetContentString(p) # Set content of the file from given string.
    text = ' '.join(p) # para trasnformaar em string
    # Create tokens - number representation of our text
    tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
    # Input tokens
    #tokens
    # Summarize 
    summary = model.generate(**tokens)
    # Output summary tokens
    #summary[0]
    # Decode summary
    #tokenizer.decode(summary[0])
    file_epub.GetContentFile('Neal_Stephenson_-_Termination_Shock_RESUMO.txt')
    update = file_epub.GetContentString() + '\n' + tokenizer.decode(summary[0])
    file_epub.SetContentString(update)
    file_epub.Upload()    

#    
#    if capitulo_rch != 0:           
#      capitulo_livro.insert(capitulo_rch, paragraphs)
#for p in capitulo_livro[1]:
#  if p != 0:
#    text = p
#    # Create tokens - number representation of our text
#    tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
#    # Input tokens
#    tokens
#    # Summarize 
#    summary = model.generate(**tokens)
#    # Output summary tokens
#    summary[0]
#    # Decode summary
#    tokenizer.decode(summary[0])